In [1]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter

# Load the report (Ensure the file is in the working directory)
pdf_name = "Att 5. Air Quality Assessment 4567 Old Northern Road Maroota.pdf"
loader = PyMuPDFLoader(pdf_name)
pages = loader.load()

# Split into manageable chunks
text_splitter = CharacterTextSplitter(
    separator="\n", chunk_size=1000, chunk_overlap=150, length_function=len
)
docs = text_splitter.split_documents(pages)

# Remove unwanted whitespaces
for doc in docs:
    doc.page_content = doc.page_content.replace("\n", "")


In [2]:
from genaicore import azure_openai_text_embeddings_llm
from langchain_community.vectorstores import FAISS

# Generate embeddings
embeddings = azure_openai_text_embeddings_llm

# Create a FAISS vector database
db = FAISS.from_documents(docs, embeddings)

# Build a retriever
retriever = db.as_retriever(search_kwargs={"k": 5})  # Retrieve top 5 relevant chunks


In [5]:
# Load environmental standards
standards_file = "Environment_standards.txt"

with open(standards_file, "r") as f:
    standards = f.read()


In [23]:
# query = """Extract the following information from the attached PDF report, starting from page 31 (onwards) under the section 'Annual Average Concentrations':

# All environment parameters such as TSP, SO2, NO2, O3, Pb, PM2.5, PM10, deposited dust, CO, and HF.

# The values for each parameter.

# The dates when these values were measured.

# Any relevant notes related to these measurements.

# Please also check for cumulative values where applicable.

# Additionally, from the section starting on page 93, extract the values for each residential location (R1 to R20) and check if any of the values mentioned under R1 to R20 exceed the following limits:

# 50 (24 hours for PM10) and 25 (Annual for PM10)

# 25 (24 hours for PM2.5) and 8 (Annual for PM2.5)

# 90 (Annual for TSP)

# 2 (Annual Increment for Dust Deposition) and 4 (Annual Cumulative for Dust Deposition)

# The units of measurement (UOM) for these parameters are as follows:

# For Dust Deposited: g/m²/day

# For others: µg/m³

# Please provide a comprehensive summary of your findings.
# """
query='''Extract the following information from the attached PDF report:

- All environmental parameters such as TSP, SO2, NO2, O3, Pb, PM2.5, PM10, deposited dust, CO, HF.
- The values for each parameter.
- The dates when these values were measured.
- Any relevant notes related to these measurements.
- Check if there are any cumulative values where applicable.

Additionally, extract the values for each residential location (R1 to R20) and check if any of the values mentioned under these locations exceed the permissible limits for the following parameters:

- PM10: 50 (24-hour limit), 25 (Annual limit)
- PM2.5: 25 (24-hour limit), 8 (Annual limit)
- TSP: 90 (Annual limit)

The units of measurement (UOM) for these parameters are as follows:

- Dust Deposited: g/m²/day
- Others: µg/m³

Please provide a comprehensive summary of your findings, including whether any values exceed the specified limits.
'''

retrieved_docs = retriever.invoke(query)
retrieved_text = "\n\n".join([doc.page_content for doc in retrieved_docs])


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from genaicore import azure_gpt4_openai_text_chat_llm
# Define LLM
llm = azure_gpt4_openai_text_chat_llm

# # Construct a structured prompt
# template = """
# You are an AI environmental analyst. Compare the retrieved environmental data from the report with the given standards.

# ### **Context: Retrieved Client Data**
# {context}

# ### **Environmental Standards**
# {standards}

# ### **Task**
# 1. Compare each environmental parameter with its respective permissible value and company name your are comparing with the standards.
# 2. Identify whether each parameter passes or fails.
# 3. Clearly indicate the retrieved value vs the standard permissible value.
# 4. Format the results in a structured table.

# ### **Example Output Format**
# | Parameter | Client Value | Standard Limit | Pass/Fail |
# |-----------|-------------|----------------|-----------|
# | TSP       | 85 µg/m³    | 90 µg/m³       | ✅ Pass   |
# | PM10      | 30 µg/m³    | 20 µg/m³       | ❌ Fail   |

# ### **Notes**
# - Only use the provided context for analysis.
# - Do not make assumptions or generate values that are not retrieved from the document.
# """

template='''You are an AI environmental analyst. Your task is to analyze the environmental report provided by a client and compare it with the given standards.

### **Client Name:**  
Extract the name of the company from the given context and display it in the report. If the company name is missing, explicitly mention: **"Client name not found in the provided data."**  

---

## **🔹 Context: Retrieved Environmental Data**  
{context}

## **🔹 Environmental Standards**  
{standards}

---

## **🔹 Analysis Task**
1. **Identify and compare environmental parameters** (e.g., TSP, PM10, PM2.5, NO2, etc.) from the report with the corresponding standard limits.
2. **Ensure unit consistency** – if the client’s values are in different units, convert them to match the standards before comparison.
3. **Clearly state whether each parameter meets the standard (Pass/Fail).**
4. **Present the comparison in a structured table format.**
5. **If any data is missing in the provided context, explicitly mention it instead of making assumptions.**

---

## **🔹Example Output Format(these are example values but you want to get thse values from the {standards}**
### **Environmental Compliance Report for [Client Name]**

| Parameter | Client Value (Converted) | Standard Limit | Pass/Fail |
|-----------|--------------------------|----------------|-----------|
| |                                                         ✅ Pass 

---

### **🔹 Additional Notes**
- If a value is missing, mention **"Data Missing"** in the table instead of assuming.
- The report strictly follows the provided **context** and **standards**.
- Any necessary unit conversions are applied before comparison.

---

### **🔹 Conclusion**
Summarize the key findings:  
- Number of **parameters that passed**.  
- Number of **parameters that failed**.  
- If any **critical violations** exist.  

If any information is missing, list the **specific missing parameters** at the end.

---
'''


# Create prompt template
prompt = ChatPromptTemplate.from_template(template)

# Construct the chain manually
formatted_prompt = prompt.invoke({"context": retrieved_text, "standards": standards})
response = llm.invoke(formatted_prompt)




In [28]:
parsed_response = StrOutputParser().invoke(response)

print(parsed_response)

### **Environmental Compliance Report for PF Formation Maroota**
Client name: PF Formation Maroota

---

### **🔹 Context: Retrieved Environmental Data**  
### **🔹 Environmental Standards**  
### **Extracted Parameters and Their Permissible Limits**

Given the data provided, the environmental parameters to analyze and compare are as follows:

- PM10
- PM2.5
- TSP (Total Suspended Particulates)
- Sulfur Dioxide (SO2)
- Oxides of Nitrogen (NOx)

### **Analyzed Parameters and Their Compliance**

| Parameter                         | Client Value                    | Standard Limit                     | Pass/Fail |
|-----------------------------------|---------------------------------|------------------------------------|-----------|
| PM10 (Annual average)             | 13 µg/m³                        | 25 µg/m³                           | Pass      |
| PM10 (24-hour maximum average)     | 50 µg/m³                        | 50 µg/m³                           | Pass      |
| PM2.5 (Annual av